In [1]:
import geopandas
SHP = r"C:\Users\rmagala\Downloads\shp.shp"
gpd = geopandas.read_file(r"C:\Users\rmagala\Downloads\shp.shp")
sample_t = gpd.sample_points(method='uniform', size=20, n_seeds=5, cluster_radius=750)

In [2]:
m = gpd.explore()
#sample_t.explore(color='red', m=m)

In [3]:
def _create_polygon_m(x, y, res_m):
    from shapely.geometry import Polygon

    return Polygon([
        (x, y),
        (x + res_m, y),
        (x + res_m, y + res_m),
        (x, y + res_m),
        (x, y)
    ])


In [4]:
from apsimNGpy.parallel.process import custom_parallel

In [5]:
def create_fishnet_km(
    shp,
    resolution_km=5,
    target_epsg=26915,
    return_lonlat=True
):
    """
    Create a clean square fishnet grid (polygons only) using a single
    resolution value in kilometers. All internal computations are in meters.
    """
    import geopandas as gpd
    import numpy as np
    from shapely.geometry import Polygon

    # km → meters
    res_m = resolution_km * 1000

    # Read input
    gdf_shape = gpd.read_file(shp)
    if gdf_shape.crs is None:
        raise ValueError("Input shapefile has no CRS defined")

    # Project to metric CRS
    gdf_proj = gdf_shape.to_crs(epsg=target_epsg)

    # Bounds in meters
    minx, miny, maxx, maxy = gdf_proj.total_bounds

    xs = np.arange(minx, maxx, res_m)
    ys = np.arange(miny, maxy, res_m)

    # Create full grid (clean squares)
    polygons = [
        Polygon([
            (x, y),
            (x + res_m, y),
            (x + res_m, y + res_m),
            (x, y + res_m),
            (x, y)
        ])
        for x in xs
        for y in ys
    ]

    gdf_grid = gpd.GeoDataFrame(
        geometry=polygons,
        crs=gdf_proj.crs
    )

    # --- IMPORTANT CLEANING STEPS ---

    # 1. Fix topology
    gdf_grid["geometry"] = gdf_grid.geometry.buffer(0)

    # 2. Clip
    gdf_clip = gpd.clip(gdf_grid, gdf_proj)

    # 3. KEEP POLYGONS ONLY (drop lines / points / collections)
    gdf_clip = gdf_clip[gdf_clip.geometry.type.isin(["Polygon", "MultiPolygon"])]

    # 4. Drop zero-area / sliver polygons
    gdf_clip = gdf_clip[gdf_clip.geometry.area > 0]

    # Optional: explode multipolygons into clean polygons
    gdf_clip = gdf_clip.explode(ignore_index=True)

   

    return gdf_clip


In [6]:
clips= create_fishnet_km(SHP, resolution_km=0.2, return_lonlat=False)

In [7]:
#clips.explore()

In [74]:
clips= create_fishnet_km(SHP, resolution_km=0.2, return_lonlat=False)
c_sample = clips.sample_points(size=10, n_seeds=5, cluster_radius=100)

In [75]:
c_sample.explore(m=clips.explore(), color='red')

In [76]:
clips

,geometry
0,"POLYGON ((431715.371 4654058.953, 431715.371 4..."
1,"POLYGON ((431715.371 4654058.953, 431915.371 4..."
2,"POLYGON ((431915.371 4654058.953, 432007.797 4..."
3,"POLYGON ((432515.371 4654258.953, 432715.371 4..."
4,"POLYGON ((432315.371 4654258.953, 432515.371 4..."
...,...
1995,"POLYGON ((441715.371 4659484.221, 441915.371 4..."
1996,"POLYGON ((441915.371 4659480.849, 442001.907 4..."
1997,"POLYGON ((441515.371 4659458.953, 441401.884 4..."
1998,"POLYGON ((441515.371 4659487.594, 441715.371 4..."


In [77]:
cc = c_sample.to_crs(epsg=4326)

In [78]:
cc

0       MULTIPOINT ((-93.82506 42.0356), (-93.82504 42...
1       MULTIPOINT ((-93.82457 42.03517), (-93.8242 42...
2       MULTIPOINT ((-93.82255 42.0353), (-93.82238 42...
3       MULTIPOINT ((-93.81505 42.03692), (-93.81457 4...
4       MULTIPOINT ((-93.81749 42.03741), (-93.81709 4...
                              ...                        
1995    MULTIPOINT ((-93.70456 42.08523), (-93.70433 4...
1996    MULTIPOINT ((-93.70218 42.08516), (-93.702 42....
1997    MULTIPOINT ((-93.70839 42.08516), (-93.70834 4...
1998    MULTIPOINT ((-93.70696 42.08516), (-93.70675 4...
1999    MULTIPOINT ((-93.70965 42.08323), (-93.70964 4...
Name: sampled_points, Length: 2000, dtype: geometry

In [79]:
points = (
    cc
    .explode(index_parts=False)
    .reset_index(drop=False)
)


In [80]:
p =[i for  i in points.sampled_points][0]

In [114]:
from logger import logger
from exceptions import ApsimRuntimeError
from apsimNGpy.core.apsim import ApsimModel
from pathlib import Path
import pandas as pd
ID ='ID'
def create_simulations(base_file: str | Path, pps:pd.DataFrame, base_simulation:str=None, index:str='index'):
    from apsimNGpy.core.model_tools import clone_simulation
    
    copy_pps = pps.copy()
    copy_pps[ID] = pps[index]
    grps = copy_pps.groupby(index)
    for k, g  in grps:
        sim_id = [f"S_{ID}_{i}_" for i in  zip(g[ID], range(g.shape[0]))]
        lon_lats = [(XY.x, XY.y) for XY in g.sampled_points]
        with ApsimModel(base_file) as model:
            model.edit_model(model_type='Models.Manager', model_name='Sow using a variable rule',
                             StartDate='28-apr',
                             EndDate='28-may',
                             Population=8.65,
                             CultivarName='B_110')
            model.edit_model(model_type='Models.Manager', model_name='Fertilise at sowing',
                             Amount=280,
                            )
            if len(model.simulations) > 1:
                raise ValueError(f'Base simulation file should have only base simulation under its node')
            base_simulation = base_simulation or model.simulations[0].Name
            new_sims_names = sim_id[:]
            new_sims_names[-1] = base_simulation
            [clone_simulation(model, sim_name=base_simulation,rename=ren) for ren in sim_id[:-1]]
            th = [50, 150, 150, 200, 200, 200, 300, 350, 400]
            model.get_weather_from_web(lonlat=lon_lats[-2], start=1986, end=2016, simulations=None, filename=f'{Path(base_file).resolve().parent}_{lon_lats[-2]}.met', source='daymet')
            
               
            try:
                weather=[model.get_soil_from_web(simulations=sim_name, lonlat=ll, source='ssurgo', 
                                        top_fom=2010, 
                                        top_urea=0.07,top_finert=0.05,
                                        thickness_sequence=th,summer_date='1-Jun', winter_date='1-Nov') 
                         for ll, sim_name in zip(lon_lats, new_sims_names)]
                model.run(verbose=False)
           
                df= model.results
                print(k, ' = ', df.Yield.mean(), df.Yield.median(), df.Yield.max())
                # mn = df.groupby('SimulationID')['Yield'].max()
                # print(mn)
            except (AttributeError, ApsimRuntimeError):
                  pass
            except ApsimRuntimeError:
                  pass
            
    
    

In [115]:
create_simulations('Maize', points)

0  =  12547.98140522119 12874.82036392568 14538.398135455916
1  =  12638.374896505447 12950.628275018045 14527.609505197604
2  =  12639.19605947935 12950.628242823996 14528.009758445716
3  =  12733.288139461576 13146.94013186574 14542.217868916458
5  =  12708.289762692099 13124.010215626062 14543.254491241723
7  =  12694.251625948622 13104.818637952305 14542.21786478001
9  =  12641.654350381845 12950.627764704372 14528.009752317033
11  =  12604.33622557093 12874.820132351133 14551.523639773812
12  =  12652.88966434662 12950.627988678918 14542.217866269946
13  =  12656.172500500184 13104.816164437303 14542.21786086636
14  =  12593.046577308685 12870.536464046852 14551.523615409124
15  =  12723.830817315755 13146.93930860142 14542.217853106193
16  =  12679.45689813232 12950.627585963979 14542.217856799924
19  =  12749.661288019497 13131.352476282671 14523.973227390547
20  =  12703.42805927164 13097.418556592012 14523.973220301115
22  =  12693.336998279236 13097.418653526403 14523.9732254

ConnectTimeout: HTTPSConnectionPool(host='sdmdataaccess.nrcs.usda.gov', port=443): Max retries exceeded with url: /Tabular/SDMTabularService.asmx (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001D765E00860>, 'Connection to sdmdataaccess.nrcs.usda.gov timed out. (connect timeout=140)'))

In [ ]:
p.x 

In [ ]:
7084.210149510337

In [ ]:
for k, g  in points.groupby('index'):
    print(g)
    break

In [ ]:
import asyncio


async def fetch_inputs(model, sim_name, ll, th, base_file):
    await asyncio.gather(
        asyncio.to_thread(
            model.get_weather_from_web,
            lonlat=ll,
            start=1986,
            end=2023,
            simulations=sim_name,
            filename=f"{Path(base_file).parent}_{ll}{sim_name}.met"
        ),
        asyncio.to_thread(
            model.get_soil_from_web,
            simulations=sim_name,
            lonlat=ll,
            source="ssurgo",
            thickness_sequence=th,
            summer_date="1-June",
            winter_date="1-Nov"
        )
    )


In [ ]:

async def create_simulations(base_file: str | Path, pps:pd.DataFrame, base_simulation:str=None, index:str='index'):
    from apsimNGpy.core.model_tools import clone_simulation
    
    copy_pps = pps.copy()
    copy_pps[ID] = pps[index]
    grps = copy_pps.groupby(index)
    for k, g  in grps:
        sim_id = [f"S_{ID}_{i}_" for i in  zip(g[ID], range(g.shape[0]))]
        lon_lats = [(XY.x, XY.y) for XY in g.sampled_points]
        with ApsimModel(base_file) as model:
            if len(model.simulations) > 1:
                raise ValueError(f'Base simulation file should have only base simulation under its node')
            base_simulation = base_simulation or model.simulations[0].Name
            new_sims_names = sim_id[:]
            new_sims_names[-1] = base_simulation
            [clone_simulation(model, sim_name=base_simulation,rename=ren) for ren in sim_id[:-1]]
            th = [50, 100, 200, 200, 200, 300, 400, 400]
            for sim_name, ll in zip(new_sims_names, lon_lats):
                await fetch_inputs(model=model, sim_name=sim_name, ll=ll, th=th, base_file=base_file)
                 
            model.run(verbose=False)
           
            df= model.results
            print(k, ' = ', df.Yield.mean(), df.Yield.median(), df.Yield.max())

In [ ]:
await create_simulations('Maize', points)